In [3]:
#importing and cleaning data, leaving what i need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
anime = pd.read_csv('data/anime-dataset-2023.csv')


In [4]:
def clean_data(anime):
    #dropping columns
    anime = anime.drop(columns=['Members', 'Scored By', 'Favorites', 'Source', 'Licensors', 'Producers', 'Premiered', 'English name','Studios', 'Duration', 'Rating', 'Status', 'Other name', 'Aired'])
    #renaming a column
    anime = anime.rename(columns={'Synopsis': 'Description'})
    return anime

anime_clean = clean_data(anime.copy())
anime_clean.head()

,anime_id,Name,Score,Genres,Description,Type,Episodes,Rank,Popularity,Image URL
0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,41.0,43,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,189.0,602,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,328.0,246,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,2764.0,1795,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,4240.0,5126,https://cdn.myanimelist.net/images/anime/7/215...


In [ ]:
#concatenating the columns together to make it easier for model to compare data
def clean_data(anime_clean):
    #first replacing and filling in data that has "unknown"
    anime_clean['Episodes'] = anime_clean['Episodes'].str.replace("UNKNOWN", "1000+", case=False, regex=False)
    #changing column type to string
    anime_clean = anime_clean.astype({'Episodes': 'string', 'Popularity': 'string', 'Score': 'string', 'Rank': 'string'})
    anime_clean['tags'] = anime_clean['Description'] + anime_clean['Genres'] + anime_clean['Type'] + anime_clean['Episodes'] + anime_clean['Popularity'] + anime_clean['Score'] + anime_clean['Rank'] 
    anime_clean = anime_clean[['anime_id','Name','tags']]

    anime_clean['tags'] = anime_clean['tags'].apply(lambda x:x.lower())
    return anime_clean

anime_clean = clean_data(anime_clean.copy())
anime_clean.head(3)


In [7]:
anime_clean.shape
anime_clean.describe

<bound method NDFrame.describe of        anime_id                             Name  \
0             1                     Cowboy Bebop   
1             5  Cowboy Bebop: Tengoku no Tobira   
2             6                           Trigun   
3             7               Witch Hunter Robin   
4             8                   Bouken Ou Beet   
...         ...                              ...   
24900     55731                      Wu Nao Monu   
24901     55732              Bu Xing Si: Yuan Qi   
24902     55733                      Di Yi Xulie   
24903     55734         Bokura no Saishuu Sensou   
24904     55735                   Shijuuku Nichi   

                                                    tags  
0      crime is timeless. by the year 2071, humanity ...  
1      another day, another bounty—such is the life o...  
2      vash the stampede is the man with a $$60,000,0...  
3      robin sena is a powerful craft user drafted in...  
4      it is the dark century and the people a

In [8]:
#making data into vector to use cosine similarity to help model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words= 'english')

vector = cv.fit_transform(anime_clean['tags']).toarray()

vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [9]:
anime_clean.head(10)

,anime_id,Name,tags
0,1,Cowboy Bebop,"crime is timeless. by the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,"another day, another bounty—such is the life o..."
2,6,Trigun,"vash the stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,robin sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,it is the dark century and the people are suff...
5,15,Eyeshield 21,"shy, reserved, and small-statured, deimon high..."
6,16,Hachimitsu to Clover,"yuuta takemoto, a sophomore at an arts college..."
7,17,Hungry Heart: Wild Striker,as the younger brother of japanese soccer star...
8,18,Initial D Fourth Stage,takumi fujiwara finally joins ryousuke and kei...
9,19,Monster,"dr. kenzou tenma, an elite neurosurgeon recent..."


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.21432946, 0.11933359, ..., 0.07955573, 0.        ,
        0.        ],
       [0.21432946, 1.        , 0.13470398, ..., 0.13470398, 0.        ,
        0.        ],
       [0.11933359, 0.13470398, 1.        , ..., 0.125     , 0.        ,
        0.        ],
       ...,
       [0.07955573, 0.13470398, 0.125     , ..., 1.        , 0.15811388,
        0.15811388],
       [0.        , 0.        , 0.        , ..., 0.15811388, 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.15811388, 1.        ,
        1.        ]])

In [13]:
anime_clean[anime_clean['Name'] == 'Trigun'].index[0]

np.int64(2)

In [15]:
#making it recommend and give position of data similarity to list 5 recommendations in a sorted list
def recommend(anime):
    index = anime_clean[anime_clean['Name'] == anime].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True , key= lambda x: x[1])
    for i in distances[1:6]:
        print(anime_clean.iloc[i[0]].Name)

In [17]:
recommend('Naruto')

Naruto (2023)
Boruto: Naruto Next Generations
Naruto: Shippuuden
Naruto: Shippuuden Movie 4 - The Lost Tower
Boruto: Naruto the Movie


In [18]:
recommend('One Piece')

One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken
One Piece: Romance Dawn Story
One Piece Film: Strong World
One Piece Movie 14: Stampede
One Piece: Oounabara ni Hirake! Dekkai Dekkai Chichi no Yume!
